In [1]:
import pandas as pd
import numpy as np
import csv
import scipy
import sys
import os
import matplotlib.pyplot as plt

%matplotlib inline

## Характеристики задачи

In [103]:
def find_features_tasks(df):
    '''
    Находит следующие характеристики для задачи:
    
    Число периодических программ
    Общая сложность периодических программ
    Общий период
    Число разделов
    Загрузка(отношение общего времени выполнения к общему периоду)
    Число интервалов
    
    Args:
        df(pd.DataFrame): Данные о постановке задачи построения статико-динамического
                          расписания
    Returns:
        int: число программ,
        int: общая сложность программ, 
        int: размер цикла планирования, 
        int: число разделов, 
        float: загрузка процессора этой задачей, 
        int: число интервалов
    '''
    Time = 1
    periods = df['period'].as_matrix()
    runtimes = df['complexity'].as_matrix()
    partitions = df['partition'].as_matrix()
    
    num_tasks = df.shape[0]
    num_part = len(np.unique(np.sort(partitions)))
    print("Num of partitions: ", num_part)
    for x in periods:
        Time = NOK(Time, x)
        
    counts = np.array([Time/x for x in periods])
    
    all_runtime = sum(runtimes[i]*counts[i] for i in range(len(df)))
    print("Complexity: ",all_runtime)
    
    intervals = set()
    intervals.add(0)
    for x in periods:
        y = x
        while(y <= Time):
            intervals.add(y);
            y += x;
            
    num_of_intervals = len(intervals) - 1;
    print("Runtime: ",Time)
    load = all_runtime / Time
    print('Load: ', load)
    print("Intervals: ", num_of_intervals)
    return tuple((num_tasks,all_runtime, Time, num_part, load, num_of_intervals))

def find_features_works(df):
    '''
    Находит следующие характеристики для задачи
    Число задач
    Общее время выполнения
    Общий период
    Число разделов
    Загрузка(отношение общего времени выполнения к общему периоду)
    Число интервалов
    '''
    
    starts = df['start'].as_matrix()
    finishes = df['finish'].as_matrix()
    runtimes = df['runtime'].as_matrix()
    partitions = df['partition'].as_matrix()
    
    num_tasks = df.shape[0]
    num_part = len(np.unique(np.sort(partitions)))
    #print("Num of partitions: ", num_part)

    Time = np.max(finishes)   
    #counts = np.array([Time/x for x in periods])
    
    all_runtime = np.sum(runtimes)
    
    #print("All_runtime: ",all_runtime)
    
    intervals = set()
    intervals.add(0)
    for x in starts:
        intervals.add(x)
    for x in finishes:
        intervals.add(x)          
    num_of_intervals = len(intervals) - 1;
    
    #print("All time: ",Time)
    
    load = all_runtime / Time
    
    average_runtime = all_runtime/ num_tasks
    #print('Load: ', load)
    #print("Число интервалов: ", num_of_intervals)
    return tuple((num_tasks, average_runtime, num_part, load, num_of_intervals))
      
    

In [119]:
def NOD(a, b):
    while a != 0 and b != 0:
        if a > b:
            a %= b
        else:
            b %= a
    return a + b


def NOK(a, b):
    return a*b / NOD(a, b)


## Генерация по распределению  

In [4]:
def create_distrib(num_tasks, max_period = 10, mode='normal', mul_period=10, part=None, load=1.):
    '''
    Создается таблица разделы х задачи, в ячейках указывается время работы, как часть от периода.
    И задается такая же табличка для периодов.
    Можно указать загрузку.

    num_tasks - вектор числа задач по разделам

    mode - normal - равномерное распределени
    '''
    if (mode == 'normal'):
        num_part = len(num_tasks)
        max_tasks = max(num_tasks)
        all_tasks = sum(num_tasks)
        distrib = np.zeros((num_part, max_tasks))
        period = np.zeros((num_part, max_tasks))
        for i in range(num_part):
            for j in range(num_tasks[i]):
                period[i][j] = int(np.random.randint(1, max_period + 1)) * mul_period
                distrib[i][j] = load*1./all_tasks
        return distrib, period
    elif (mode == 'one_part'):
        num_part = num_tasks.shape[0]
        max_tasks = num_tasks.max()
        all_tasks = num_tasks.sum() - 1
        distrib = np.zeros((num_part, max_tasks))
        period = np.zeros((num_part, max_tasks))
        for i in range(num_part):
            for j in range(num_tasks[i]):
                period[i][j] = int(np.random.randint(1, max_period + 1)) * mul_period
                distrib[i][j] = load*(1.-part)/all_tasks
        period[0][0] = max_period * mul_period
        distrib[0][0] = part * load
        return distrib, period

def create_tasks_by_distrib(task_distrib, periods):
    '''
    Create one task for static-dynamic shedule
    task_distrib - nparray
    first axis - partition, second axis - distribution of time for every
    periods - periods for tasks
    '''
    df = pd.DataFrame(columns=['partition', 'runtime', 'period'])
    for i in range(task_distrib.shape[0]):
        for j in range(task_distrib.shape[1]):
            if (task_distrib[i][j] == 0):
                break
                
            d = np.array([0, 0, 0], dtype=int)
            d[0] = i + 1
            d[2] = periods[i][j]
            d[1] = task_distrib[i][j] * d[2]
            if (d[1] <= 1.):
                d[1] = 1.
            df.loc[len(df)] = d
    return df
    

example_distrib = np.array([[0.1, 0.3, 0.2],
                            [0.1, 0, 0],
                            [0.2, 0 ,0]])
example_periods = np.array([[10, 20, 40],
                            [50, 0, 0],
                            [100, 0 ,0]])

def create_test_sample_dist(num_tasks, DIR, mul_period=10, mode='normal', part=None, max_period = 10, load=1., time_to_switch = 1):
    '''
    num_tasks - список массивов
    Создает len(num_tasks) файлов с именами testi.tsv, где i - номер задачи
    В файл meta.tsv помещается информация об этих задачах
    Возвращает строку с именами файлов, записанными через запятую
    '''

    os.mkdir(DIR)
    with open(DIR+'/TEST.txt', 'w') as file_output:
        df_f =  pd.DataFrame(columns=['num_tasks','all_runtime', 'Time', 'num_part', 'load', 'num_of_intervals', 'time_to_switch'])
        for x in range(num_tasks.shape[0]):
            file = DIR+"/test" + str(x) + ".tsv"
            file_output.write(file+' '+str(time_to_switch)+'\n');
            print (num_tasks[x])
            distr, period = create_distrib(num_tasks[x], mul_period=mul_period, mode=mode, part=part, max_period=max_period, load=load)
            df = create_tasks_by_distrib(distr, period)
            df.to_csv(file, sep ="\t")

            features = find_features_tasks(df)       
            node = np.empty((len(features))+1)
            for i in range(len(features)):
                node[i] = features[i]
            node[len(features)] = time_to_switch
            df_f.loc[len(df_f)] = node 
        
    df_f.to_csv(DIR+"/meta.tsv", sep="\t")

tasks = np.array([[6, 2, 3],
                  [1, 2, 4],
                  [1, 2]])

create_test_sample_dist(tasks, "TestXIIIIII", mul_period=1000, mode='normal', max_period=10, load=0.7)

---

# Периодические работы

## Равномерное распределение параметров задач

In [120]:
def create_tasks_random(n_tasks, n_part, periods, load):
    '''
    Create one task for static-dynamic shedule с периодическими интервалами 
    по равномерному распределению
    Args:
        n_tasks(int): Число периодических программ в задаче
        n_part(int): число разделов в задаче
        periods(list(int)): возможные значения для периодов программ
        load: приблизительная нагрузка полученной задачи
    
    Returns:
        pd.DataFrame: Постановка задачи в табличном формате с колонками
                      'partition', 'complexity', 'period', 'functionalities','cost'
    '''
    df = pd.DataFrame(columns=['partition', 'complexity', 'period', 'functionalities','cost'])
    parts = np.array(range(1, n_part + 1))
    np.random.shuffle(parts)
    for i in range(n_tasks):
        d = []
        d.append( parts[i%n_part])
        np.random.shuffle(periods)
        d.append( np.random.randint(1, (1.8 * periods[0] / (n_tasks/load)) + 1))
        d.append(periods[0] )
        d.append('f1')
        d.append(0)
        df.loc[len(df)] = d
    return df
    

In [185]:
df_test = create_tasks_random(n_tasks=6, n_part=3, periods=[50, 100, 200], load=3)

In [186]:
df_test

,partition,complexity,period,functionalities,cost
0,2,12,50,f1,0
1,1,1,100,f1,0
2,3,177,200,f1,0
3,2,32,50,f1,0
4,1,44,100,f1,0
5,3,9,100,f1,0


In [187]:
find_features_tasks(df_test)

Num of partitions:  3
Complexity:  461.0
Runtime:  200.0
Load:  2.305
Intervals:  4


/Users/as.smirnov/miniconda3/envs/mt/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
/Users/as.smirnov/miniconda3/envs/mt/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/Users/as.smirnov/miniconda3/envs/mt/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(6, 461.0, 200.0, 3, 2.305, 4)

In [172]:
df_test.loc[4, 'complexity'] = 15

In [188]:
df_test.to_csv('C++/TestsHeterogenes/test_0.csv', sep='\t')

 -----------------
# Обычные работы


## Равномерное распределение

In [107]:
def create_works_plan(max_time, delta, num_of_works, num_of_part, load, c, n_proc=1, mul=1):
    '''
    Создание планируемого расписания  
    '''
    df = pd.DataFrame(columns=['partition', 'runtime', 'start', 'finish'])
    for j in range(n_proc):
        current_time = 0  
        for i in range(int(num_of_works/n_proc)):
            d = np.array([0, 0, 0, 0], dtype=int)
            d[1] = np.random.randint(1,max_time)*mul
            d[0] = np.random.randint(0, num_of_part/n_proc) + j*(num_of_part/n_proc) + 1
            d[2] = max(current_time - np.random.randint(delta)*mul, 0)
            d[3] = current_time + d[1] + np.random.randint(delta)*mul
            current_time += c + d[1]*1.0/load
            df.loc[len(df)] = d
    return df

In [112]:
df_test = create_works_plan(max_time=10, delta=100, num_of_works=37, num_of_part=7, load=1, c=1)

In [113]:
df_test

,partition,runtime,start,finish
0,1,2,0,20
1,7,9,0,62
2,7,4,0,113
3,1,9,0,104
4,2,5,0,83
5,2,4,0,55
6,6,7,0,48
7,1,2,46,62
8,6,1,0,79
9,3,6,22,135


In [114]:
def create_works_random(num_works, num_part, max_time):
    '''
    равномерное распределение
    для работ
    
    '''
    df = pd.DataFrame(columns=['partition', 'runtime', 'start', 'finish'])
    for i in range(num_works):
        d = np.array([0, 0, 0, 0], dtype=int)
        d[0] = np.random.randint(1, num_part + 1)
        d[2] = np.random.randint(1, max_time + 1)
        d[3] = np.random.randint(d[2], max_time + 1)
        d[1] = np.random.randint(d[2], d[3] + 1) - d[2]
        df.loc[len(df)] = d
    return df

In [117]:
df_test = create_works_random(num_works=37, num_part=7, max_time=100)

In [118]:
df_test

,partition,runtime,start,finish
0,3,6,23,86
1,5,2,94,98
2,6,6,60,72
3,6,11,57,72
4,2,6,73,79
5,2,15,52,90
6,1,0,93,96
7,1,1,82,99
8,1,3,49,55
9,5,11,59,75


---

# Обработка решения

In [105]:
def get_time_perfomance(file):
    time = 0.
    perfomance = 0.
    with open(file, 'r') as f:
        str_ = f.readline();
        time = float(str_.split(' ')[1])
        str_ = f.readline();
        #c = float(str_.split(' ')[1])
        str_ = f.readline();
        perfomance = float(str_.split(' ')[1])
    return time, perfomance

In [106]:
def result_to_df(files):
    df = pd.DataFrame(columns=['Time_alg','Effectiveness'])
    for file in files:
        node = np.array([0,0], dtype=float)
        node[0], node[1] = get_time_perfomance(file)
        df.loc[len(df)] = node
    return df

In [69]:
def save(name='', fmt='png'):
    pwd = os.getcwd()
    iPath = './pictures/{}'.format(fmt)
    if not os.path.exists(iPath):
        os.mkdir(iPath)
    os.chdir(iPath)
    plt.savefig('{}.{}'.format(name, fmt), fmt='png')
    os.chdir(pwd)
    #plt.close()